# Imports

In [1]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 72.6 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
import numpy as np
import pandas as pd
import matplotlib
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import optuna

rng = np.random.RandomState(42)

In [3]:
import sklearn
sklearn.__version__

'1.6.0'

# Data

In [4]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# Baseline

In [5]:
model = HistGradientBoostingRegressor(
    max_iter=10000, 
    early_stopping=True, validation_fraction=0.1, n_iter_no_change=25,
    verbose=1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Binning 0.013 GB of training data: 0.152 s
Binning 0.001 GB of validation data: 0.008 s
Fitting gradient boosted rounds:
Fit 129 trees in 1.734 s, (3999 total leaves)
Time spent computing histograms: 0.797s
Time spent finding best splits:  0.257s
Time spent applying splits:      0.174s
Time spent predicting:           0.020s
Train MAE: 2.6490688741135675
Valid MAE: 3.134263657064129


# Hyperparameter tuning

In [6]:
def load_data():
    data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
    data = pd.read_csv(data_path)
    data['DATE'] = data['DATE'].astype('datetime64[ns]')
    X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
    y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
    X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
    y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
    return X_train, X_valid, y_train, y_valid

# max_iter is determined by early stopping
def objective(trial):
    PARAMS = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 0.5),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 64),
        'max_depth': trial.suggest_int('max_depth', 1, 12),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 1000),
        'l2_regularization': trial.suggest_float('l2_regularization', 0.0, 100),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0)
    }
    X_train, _, y_train, _ = load_data()
    model = HistGradientBoostingRegressor(
        max_iter=10000, 
        early_stopping=True, validation_fraction=0.1, n_iter_no_change=25,
        verbose=0, random_state=rng
    )
    model.set_params(**PARAMS)
    score = np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error'))
    return score

In [7]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials = 50, # the random sampling is used instead of the TPE algorithm until the given number of trials finish in the same study
    n_ei_candidates = 24, # number of candidate samples used to calculate the expected improvement.
    multivariate = True, # multivariate TPE when suggesting candidates; default: False
    seed = 42
)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=100, interval_steps=20),
    direction='maximize',
    sampler=sampler,
    storage='sqlite:///db.sqlite3'
)
study.optimize(
    objective, n_trials = 1000,
    timeout = 3600*10, # in seconds
    n_jobs = 2,
    show_progress_bar = True
)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-12-21 07:51:19,687] A new study created in RDB with name: no-name-53f0b40f-ca29-43e5-a2e6-69d0ef7b67b8


  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2024-12-21 07:51:26,025] Trial 0 finished with value: -3.062051568820993 and parameters: {'learning_rate': 0.11282487317001402, 'max_leaf_nodes': 60, 'max_depth': 9, 'min_samples_leaf': 787, 'l2_regularization': 17.012604928864206, 'max_features': 0.5424692127495221}. Best is trial 0 with value: -3.062051568820993.
[I 2024-12-21 07:51:26,948] Trial 1 finished with value: -3.0893272053563394 and parameters: {'learning_rate': 0.16203150954516554, 'max_leaf_nodes': 27, 'max_depth': 9, 'min_samples_leaf': 892, 'l2_regularization': 98.03711752076059, 'max_features': 0.14296519443072442}. Best is trial 0 with value: -3.062051568820993.
[I 2024-12-21 07:51:29,422] Trial 2 finished with value: -3.0331376049800975 and parameters: {'learning_rate': 0.26286038607764706, 'max_leaf_nodes': 12, 'max_depth': 4, 'min_samples_leaf': 248, 'l2_regularization': 53.2786886406137, 'max_features': 0.8839843701101604}. Best is trial 2 with value: -3.0331376049800975.
[I 2024-12-21 07:51:30,799] Trial 3 fin

In [9]:
print(f"Best score: {study.best_trial.value}")
print(f"Best params: {study.best_trial.params}")

Best score: -2.9774395944624477
Best params: {'learning_rate': 0.029939722460644722, 'max_leaf_nodes': 14, 'max_depth': 1, 'min_samples_leaf': 13, 'l2_regularization': 4.530169807586777, 'max_features': 0.9644307562332378}


In [11]:
model = HistGradientBoostingRegressor(
    max_iter=10000, 
    learning_rate=0.03, max_leaf_nodes=14, max_depth=1,
    min_samples_leaf=13, l2_regularization=4.53, max_features=0.964,
    early_stopping=True, validation_fraction=0.1, n_iter_no_change=250,
    verbose=1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Binning 0.013 GB of training data: 0.147 s
Binning 0.001 GB of validation data: 0.006 s
Fitting gradient boosted rounds:
Fit 10000 trees in 33.115 s, (20000 total leaves)
Time spent computing histograms: 23.085s
Time spent finding best splits:  1.250s
Time spent applying splits:      0.919s
Time spent predicting:           0.659s
Train MAE: 2.882869657909171
Valid MAE: 3.1453964152058465


In [13]:
model = HistGradientBoostingRegressor(
    max_iter=10000, 
    learning_rate=0.03, max_leaf_nodes=14, max_depth=1,
    min_samples_leaf=15, l2_regularization=4.5, max_features=1.0,
    early_stopping=True, validation_fraction=0.1, n_iter_no_change=250,
    verbose=1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Binning 0.013 GB of training data: 0.171 s
Binning 0.001 GB of validation data: 0.007 s
Fitting gradient boosted rounds:
Fit 9470 trees in 31.118 s, (18940 total leaves)
Time spent computing histograms: 21.798s
Time spent finding best splits:  0.736s
Time spent applying splits:      1.002s
Time spent predicting:           0.623s
Train MAE: 2.882107895972987
Valid MAE: 3.1493315995685025
